# **Data PreProcessing**
Preprocessing data before extracting haunted features. 

Descriptions Parsed using [tika](https://github.com/chrismattmann/tika-python) and [org.apache.tika.parser.DefaultParser]



## **Clean Stop Words and Punctuation**
We found that datefinder and number parser work best when strings are cleaned of stopwords.

1. Stopwords cleaned using [natural_language_toolkit_python](https://www.nltk.org/)
    - Stopword Set: nltk.corpus.stopwords
2. Punctuations removed using python [string_module](https://docs.python.org/3/library/string.html)
    - Punctuation Set: {!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~.}


In [82]:
import tika as tk
from tika import parser
import pandas as pd

## Extract Text from File ##

def extract_text(file):
    # Parse the CSV file
    parsed_data = tk.parser.from_file(file)

    # Extract Text 
    csv_text = parsed_data["content"]
    # Print Metadata
    print(parsed_data["metadata"])

    return csv_text

extracted_text = extract_text("../data/raw/haunted_places.tab")
extracted_text_list = extracted_text.split("\t")

## Read DataFrame Using Pandas to Get Column Index ##
df = pd.read_csv("../data/raw/haunted_places.tab", delimiter = "\t") 
headers = df.columns.tolist()
column_idx = headers.index("description") + 1
num_rows = df.shape[0]

## Extract Descriptions from List, Set all Chars to Lowercase ##
extracted_descriptions = [extracted_text_list[column_idx + i * len(headers)] for i in range(1,num_rows + 1)]
extracted_descriptions = list(map(lambda x : x.lower().strip(), extracted_descriptions))

{'X-TIKA:Parsed-By': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.csv.TextAndCSVParser'], 'X-TIKA:Parsed-By-Full-Set': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.csv.TextAndCSVParser'], 'X-TIKA:content_handler': 'ToTextContentHandler', 'Content-Encoding': 'UTF-8', 'X-TIKA:parse_time_millis': '130', 'X-TIKA:embedded_depth': '0', 'resourceName': "b'haunted_places.tab'", 'csv:delimiter': 'tab', 'Content-Length': '5352328', 'Content-Type': 'text/tsv; charset=UTF-8; delimiter=tab'}


In [96]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

## Download StopWords and Tokenizer ##
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words("english"))


## init Clean Descriptions##
cleaned_descriptions = []
df_out = df.copy()

for i in range(len(extracted_descriptions)):
    description = extracted_descriptions[i]

    ## Only Add Non-Stopwords and Non-Punctuation Tokens ##
    description = " ".join([word for word in word_tokenize(description) if (word not in stop_words) and (word not in string.punctuation)])

    ## Write Cleaned Description To df_out ##
    df_out.iloc[i, column_idx - 1] = description


## Fill NAN Values ##
df_out["description"] = df_out["description"].fillna("Invalid_Description").astype(str)
df_out.to_csv("../data/processed/haunted_places_cleaned.tab", index = False, header = True, sep = "\t")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dgottschalk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dgottschalk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [94]:
type(df_out.loc[1063,"description"])

str